<a href="https://www.kaggle.com/code/buketzdamar/e-commerce?scriptVersionId=222733171" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
data = pd.read_csv("/kaggle/input/e-commerce-datasets/data.csv", encoding='unicode_escape')

**InvoiceNo: Invoice Number**  
A unique invoice number for each transaction. This identifies a particular sale and may include an invoice containing multiple products.

**StockCode: Product Code**  
A unique code for the product. It is used to track the product's stock and variety.

**Description: Product Description**  
The name or description of the product. It provides text explaining what the product is.

**Quantity: Quantity**  
The number of items purchased. This specifies the quantity sold.

**InvoiceDate: Invoice Date**  
The date and time when the invoice was issued. This can be used to analyze the timing of sales and seasonal trends.

**UnitPrice: Unit Price**  
The selling price of the product. It represents the price of a single unit of the product.

**CustomerID: Customer ID**  
A unique identification number for each customer. This is important for tracking each customer's transaction history.

**Country: Country**  
The country where the customer is located. This information is used for geographical analyses or market segmentation.


In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
# data information
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# missing values

print(data.isnull().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [6]:
print(data['Description'])

0          WHITE HANGING HEART T-LIGHT HOLDER
1                         WHITE METAL LANTERN
2              CREAM CUPID HEARTS COAT HANGER
3         KNITTED UNION FLAG HOT WATER BOTTLE
4              RED WOOLLY HOTTIE WHITE HEART.
                         ...                 
541904            PACK OF 20 SPACEBOY NAPKINS
541905           CHILDREN'S APRON DOLLY GIRL 
541906          CHILDRENS CUTLERY DOLLY GIRL 
541907        CHILDRENS CUTLERY CIRCUS PARADE
541908          BAKING SET 9 PIECE RETROSPOT 
Name: Description, Length: 541909, dtype: object


In [7]:
# Let's drop rows with missing CustomerID values

data = data.dropna(subset=['CustomerID', 'Description'])


In [8]:
print(data.isnull().sum())

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64


In [9]:
data.count

<bound method DataFrame.count of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

            InvoiceDate  UnitPrice  Custom

In [10]:
# Summary statistics for numerical columns

print(data.describe())


            Quantity      UnitPrice     CustomerID
count  406829.000000  406829.000000  406829.000000
mean       12.061303       3.460471   15287.690570
std       248.693370      69.315162    1713.600303
min    -80995.000000       0.000000   12346.000000
25%         2.000000       1.250000   13953.000000
50%         5.000000       1.950000   15152.000000
75%        12.000000       3.750000   16791.000000
max     80995.000000   38970.000000   18287.000000


In [11]:
data.describe(include='all').T # Operations are performed not only on numerical data but also on categorical data.

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
InvoiceNo,406829,22190,576339,542,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StockCode,406829,3684,85123A,2077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,406829,3896,WHITE HANGING HEART T-LIGHT HOLDER,2070,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,406829.0,NaN,NaN,NaN,12.061303,248.69337,-80995.0,2.0,5.0,12.0,80995.0
InvoiceDate,406829,20460,11/14/2011 15:27,543,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UnitPrice,406829.0,NaN,NaN,NaN,3.460471,69.315162,0.0,1.25,1.95,3.75,38970.0
CustomerID,406829.0,NaN,NaN,NaN,15287.69057,1713.600303,12346.0,13953.0,15152.0,16791.0,18287.0
Country,406829,37,United Kingdom,361878,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Convert the invoice date to datetime format

data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [14]:
# Sales revenue calculation: Quantity * Unit Price

data['Sales'] = data['Quantity'] * data['UnitPrice']


In [15]:
# Total revenue

total_sales = data['Sales'].sum()
print(f"Toplam Satış Geliri: {total_sales}")


Toplam Satış Geliri: 8300065.813999999


In [16]:
# List best-selling products

top_products = data.groupby('Description').agg({'Sales': 'sum'}).sort_values(by='Sales', ascending=False)
print(top_products)


                                        Sales
Description                                  
REGENCY CAKESTAND 3 TIER            132870.40
WHITE HANGING HEART T-LIGHT HOLDER   93823.85
JUMBO BAG RED RETROSPOT              83236.76
PARTY BUNTING                        67687.53
POSTAGE                              66710.24
...                                       ...
WHITE CHERRY LIGHTS                    -54.00
CHALKBOARD KITCHEN ORGANISER           -87.80
Discount                             -5696.22
CRUK Commission                      -7933.43
Manual                              -58385.46

[3896 rows x 1 columns]


In [17]:
# Sales by Country

c_sales = data.groupby('Country').agg({'Sales': 'sum'}).sort_values(by='Sales', ascending=False)
print(c_sales)

                            Sales
Country                          
United Kingdom        6767873.394
Netherlands            284661.540
EIRE                   250285.220
Germany                221698.210
France                 196712.840
Australia              137077.270
Switzerland             55739.400
Spain                   54774.580
Belgium                 40910.960
Sweden                  36595.910
Japan                   35340.620
Norway                  35163.460
Portugal                29059.810
Finland                 22326.740
Channel Islands         20086.290
Denmark                 18768.140
Italy                   16890.510
Cyprus                  12946.290
Austria                 10154.320
Singapore                9120.390
Poland                   7213.140
Israel                   6994.250
Greece                   4710.520
Iceland                  4310.000
Canada                   3666.380
Unspecified              2667.070
Malta                    2505.470
United Arab Em

In [18]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [19]:
# Top customer by CustomerID based on purchase frequency

best_customer = data.groupby('CustomerID').agg({'Sales': 'sum'}).sort_values(by='Sales', ascending=False)

print(best_customer)

                Sales
CustomerID           
14646.0     279489.02
18102.0     256438.49
17450.0     187482.17
14911.0     132572.62
12415.0     123725.45
...               ...
12503.0      -1126.00
17603.0      -1165.30
14213.0      -1192.20
15369.0      -1592.49
17448.0      -4287.63

[4372 rows x 1 columns]


In [20]:
# Date with most transactions

date_sales = data.groupby('InvoiceDate').agg({'Sales':'sum'}).sort_values(by='Sales', ascending=False)
print(date_sales)

                         Sales
InvoiceDate                   
2011-12-09 09:15:00  168469.60
2011-01-18 10:01:00   77183.60
2011-06-10 15:28:00   39597.40
2011-09-20 11:05:00   31698.16
2011-06-15 13:37:00   23426.81
...                        ...
2011-10-11 11:10:00  -11816.64
2011-04-18 13:08:00  -22998.40
2011-06-10 15:31:00  -38970.00
2011-01-18 10:17:00  -77183.60
2011-12-09 09:27:00 -168469.60

[20460 rows x 1 columns]


In [21]:
# Return analysis

returns = data[data['Quantity'] < 0]
return_details = returns.groupby(['CustomerID', 'StockCode']).agg({'Sales': 'sum', 'Quantity': 'sum'}).reset_index()

return_details_sorted = return_details.sort_values(by='Sales', ascending=False)
print(return_details_sorted)


      CustomerID StockCode      Sales  Quantity
4914     15555.0     85204      -0.12        -1
3499     14606.0    85194S      -0.19        -1
7242     17722.0     21883      -0.19        -1
7466     17841.0     84929      -0.19        -1
4892     15555.0     16235      -0.21        -1
...          ...       ...        ...       ...
409      12536.0         M   -8366.56       -22
848      12744.0         M  -12158.90        -7
4273     15098.0         M  -38970.00        -1
0        12346.0     23166  -77183.60    -74215
5878     16446.0     23843 -168469.60    -80995

[7792 rows x 4 columns]


In [22]:
# Customer with the most returns

customer_returns = returns.groupby('CustomerID').agg({'Sales': 'sum', 'Quantity': 'sum'}).sort_values(by='Quantity', ascending=False)
print(customer_returns)


                Sales  Quantity
CustomerID                     
17175.0        -12.75        -1
13811.0       -222.75        -1
17065.0       -112.35        -1
15589.0        -21.95        -1
15590.0        -30.19        -1
...               ...       ...
16029.0     -30032.23     -8060
15749.0     -22998.40     -9014
15838.0       -292.32     -9361
12346.0     -77183.60    -74215
16446.0    -168469.60    -80995

[1589 rows x 2 columns]


In [23]:
# To analyze the distribution of returns over time

returns['Date'] = returns['InvoiceDate'].dt.date
return_by_date = returns.groupby('Date').agg({'Sales': 'sum', 'Quantity': 'sum'}).sort_values(by='Quantity', ascending=False)
print(return_by_date)


                Sales  Quantity
Date                           
2011-02-28      -1.95        -1
2011-04-17     -21.20        -2
2011-07-17     -19.90        -2
2011-04-15      -6.99        -3
2011-08-28     -20.25        -4
...               ...       ...
2011-10-11  -14231.95     -7824
2011-04-18  -23840.57     -9248
2010-12-02   -1541.10    -10287
2011-01-18  -77266.55    -74236
2011-12-09 -168789.07    -81030

[302 rows x 2 columns]


<ipython-input-23-87538b18f786>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returns['Date'] = returns['InvoiceDate'].dt.date


In [24]:
data.nunique()

InvoiceNo      22190
StockCode       3684
Description     3896
Quantity         436
InvoiceDate    20460
UnitPrice        620
CustomerID      4372
Country           37
Sales           4041
dtype: int64

In [25]:
data2 = data

In [26]:
data2.to_csv('data2.csv', index=False) ## it has not missing value


In [27]:
# Cancelled orders

data2['order_canceled'] = data2['InvoiceNo'].apply(lambda x:int('C' in x))
display(data2[:5])

n1 = data2['order_canceled'].sum()
n2 = data2.shape[0]
print('Number of orders canceled: {}/{} ({:.2f}%) '.format(n1, n2, n1/n2*100))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,order_canceled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,0


Number of orders canceled: 8905/406829 (2.19%) 


In [28]:
# Countries with the most cancellations

c_country = data2.groupby('Country').agg({'order_canceled': 'sum'}).sort_values(by='order_canceled', ascending=False)
print(c_country)

                      order_canceled
Country                             
United Kingdom                  7533
Germany                          453
EIRE                             247
France                           149
USA                              112
Australia                         74
Spain                             48
Italy                             45
Belgium                           38
Japan                             37
Switzerland                       35
Portugal                          18
Malta                             15
Norway                            14
Sweden                            11
Poland                            11
Finland                           10
Channel Islands                   10
Denmark                            9
Cyprus                             8
Netherlands                        8
Singapore                          7
Czech Republic                     5
Austria                            3
Israel                             2
S

In [29]:
stok_inf = data2[data2['StockCode'].str.contains('^[a-zA-Z]+', regex=True)]['StockCode'].unique()
stok_inf

array(['POST', 'D', 'C2', 'M', 'BANK CHARGES', 'PADS', 'DOT', 'CRUK'],
      dtype=object)

POST            -> POSTAGE                       
D               -> Discount                      
C2              -> CARRIAGE                      
M               -> Manual                        
BANK CHARGES    -> Bank Charges                  
PADS            -> PADS TO MATCH ALL CUSHIONS    
DOT             -> DOTCOM POSTAGE     

In [30]:
data2.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,order_canceled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,0
